In [ ]:
import glob
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import os

import scanpy as sc
import seaborn as sns

from scroutines import basicu

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import ValueWarning
from tqdm import tqdm

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/annot/gencode.vM25.genebody.bed'
df_annot = pd.read_csv(f, sep='\t', header=None)
df_annot

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_gene_list_250409.csv'
df_annot_abc = pd.read_csv(f)
df_annot_abc['cond'] = df_annot_abc['cond'].apply(lambda x: x.replace('NR', ''))
df_annot_abc = df_annot_abc.groupby('gene').agg({
    'cond': lambda x: ' '.join(x), 
    'archetype': lambda x: ''.join(np.unique(x)), 
    })
df_annot_abc

In [ ]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_nrdr_lmm/'
!ls $ddir

In [ ]:
    
filename = os.path.join(ddir, 'NRDR_DEGs_LMM_P21.csv')
# f = os.path.join(ddir, 'P8_L23.csv')
df = pd.read_csv(filename, index_col=0)
df['max_cp10k'] = np.max(df.iloc[:,4:], axis=1)
x = df['log2fc']
y = df['qval']
z = df['max_cp10k']

x_th = np.log2(2)
y_th = 0.05
z_th = 0.1

cond_sig = np.all([np.abs(x) > x_th, y < y_th, z > z_th], axis=0)


print(np.sum(cond_sig))
df_sig = df[cond_sig].copy().sort_values('qval')
df_sig = df_sig.set_index('gene').join(df_annot_abc)
df_sig

In [ ]:
# df_sig_abc = df_sig.set_index('gene').join(df_annot_abc).dropna().sort_values('archetype')
df_sig_abc = df_sig.sort_values('archetype').dropna()
print(df_sig_abc.shape)
df_sig_abc

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, -np.log10(y), s=5)
for idx, row in df[cond_sig].iterrows():
    ax.text(row['log2fc'], -np.log10(row['qval']), row['gene'])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, -np.log10(y), s=1, color='gray')
ax.scatter(x[cond_sig], -np.log10(y)[cond_sig], s=5)
for idx, row in df[cond_sig].iterrows():
    if np.abs(row['log2fc']) < 4 and -np.log10(row['qval']) < 10:
        ax.text(row['log2fc'], -np.log10(row['qval']), row['gene'])
# ax.set_xlim([-4,4])
ax.set_ylim([-0.1,10])
# ax.axhline(-np.log10(y_th))
# ax.axvline( x_th)
# ax.axvline(-x_th)
plt.show()

In [ ]:
df.set_index('gene').loc['Scg2']

In [ ]:
df.set_index('gene').loc['Nptx2']